In [7]:
import scanpy as sc
import stlearn as st
import numpy as np
import csv
import pickle
from scipy import sparse
import scipy.io as sio
import matplotlib
matplotlib.use('Agg')
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap, to_hex, rgb2hex
from typing import List
import qnorm
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from collections import defaultdict
import pandas as pd
import gzip
from kneed import KneeLocator
import copy 
#sys.path.append("/home/gw/code/utility/altairThemes/")
#if True:  # In order to bypass isort when saving
#    import altairThemes
#import altairThemes
import altair as alt
import argparse

spot_diameter = 89.43 #pixels

In [8]:
data_name = 'PDAC_64630'

In [9]:
adata_h5 = st.Read10X(path="/Users/victoriagao/local_docs/NEST/input", count_file='filtered_feature_bc_matrix.h5')
print(adata_h5)

AnnData object with n_obs × n_vars = 1406 × 36601
    obs: 'in_tissue', 'array_row', 'array_col', 'sum_counts', 'imagecol', 'imagerow'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'


/Users/victoriagao/anaconda3/envs/NEST_try/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/victoriagao/anaconda3/envs/NEST_try/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [14]:
sc.pp.filter_genes(adata_h5, min_cells=1)
adata_h5

AnnData object with n_obs × n_vars = 1406 × 19523
    obs: 'in_tissue', 'array_row', 'array_col', 'sum_counts', 'imagecol', 'imagerow'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    uns: 'spatial'
    obsm: 'spatial'

In [13]:
gene_ids = list(adata_h5.var_names)
gene_ids

['AL627309.1',
 'AL627309.5',
 'AP006222.2',
 'LINC01409',
 'LINC01128',
 'LINC00115',
 'FAM41C',
 'LINC02593',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'AL645608.7',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'LINC01342',
 'AL390719.2',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'AL162741.1',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20-AS1',
 'MRPL20',
 'AL391244.2',
 'ANKRD65',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'AL645728.1',
 'FNDC10',
 'AL691432.2',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'SLC35E2A',
 'NADK',
 'GNB1',
 'AL109917.1',
 'CALML6',
 'TMEM52',
 'AL391845.2',
 'GABRD',
 'PRKCZ',
 'PRKCZ-AS1',
 'FAAP20',
 'SKI',
 'AL590822.3',
 'MORN1',
 'AL589739.1',
 'AL513477.2',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'AL139246.5',
 'TNFRSF14-AS1',
 'TNFRSF14',
 'AL139246.3',
 'PRXL2B',
 'MMEL1',
 'TTC34',
 'AC

In [15]:
coordinates = adata_h5.obsm['spatial']
coordinates

array([[ 9283, 12018],
       [11004, 10622],
       [ 8053,  6963],
       ...,
       [12040, 10617],
       [ 9801, 12115],
       [ 3230,  9762]])

In [22]:
cell_barcode = np.array(adata_h5.obs.index)
temp = adata_h5.X

In [23]:
## get the cell vs gene matrix ##################
temp = qnorm.quantile_normalize(np.transpose(sparse.csr_matrix.toarray(temp)))
adata_X = np.transpose(temp)

cell_vs_gene = copy.deepcopy(adata_X)

In [24]:
adata_X

array([[0.04374796, 0.04374796, 0.04374796, ..., 0.04374796, 0.04374796,
        0.04374796],
       [0.05297594, 0.05297594, 0.05297594, ..., 0.05297594, 0.05297594,
        0.05297594],
       [0.07199929, 0.07199929, 0.07199929, ..., 0.07199929, 0.07199929,
        0.07199929],
       ...,
       [0.03997147, 0.03997147, 0.03997147, ..., 0.03997147, 0.03997147,
        0.03997147],
       [0.03324644, 0.03324644, 0.03324644, ..., 0.03324644, 0.03324644,
        0.03324644],
       [0.04670132, 0.04670132, 0.04670132, ..., 0.04670132, 0.04670132,
        0.04670132]], dtype=float32)

In [25]:
##################### make cell metadata: barcode_info ###################################
 
i=0
barcode_serial = dict()
for cell_code in cell_barcode:
    barcode_serial[cell_code] = i
    i = i + 1

i=0
barcode_info=[]
for cell_code in cell_barcode:
    barcode_info.append([cell_code, coordinates[i,0],coordinates[i,1], 0]) # last entry will hold the component number later
    i=i+1

In [ ]:
####### load annotations ##############################################

if data_name == 'PDAC_64630':
    pathologist_label_file='/cluster/home/t116508uhn/IX_annotation_artifacts.csv' #IX_annotation_artifacts.csv' #
    pathologist_label=[]
    with open(pathologist_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        for line in csv_file:
            pathologist_label.append(line)	
    	
    barcode_type=dict() # record the type (annotation) of each spot (barcode)
    for i in range (1, len(pathologist_label)):
        barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]